- mu_I only zero for # e- = 0
- no frozen core approximation/pseudopotentials (pp only for pbc)
- h_core in code = e_kin + e_nuc_e


- N2 as reference system: E(\lambda) = diatomics from H2 to N2 (no basis set, core potential etc. problem)
- N2 from diatomics 3.56 Ha to low, N2 from APDFT implementation differs 230mH from N2 from diatomics

- N2 mean of lumo homo very close to N2 reference (coincidence, for qm9 molecule both are bad)

- deviation from true energy might be systematic -> could be corrected by using atomisation energies where energies of atoms are calculated from alchemy

In [ ]:
import numpy as np
import scipy as sc
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.data import nist
import qml

angstrom = 1 / 0.52917721067

In [83]:
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
total_elec = com.nuclear_charges.sum()
num_elecs = [22, 30, 38, 44, 52]
alchemical_potentials = []
e_homo = []
e_lumo = []

for num_elec in num_elecs:
    print(f'Calculating {num_elec}')
    lam_val = num_elec/total_elec
    # core_elec = com.nuclear_charges.sum() - total_num_ve

    mol = pyscf.gto.Mole()
    for ch, coords in zip(com.nuclear_charges, com.coordinates):
        mol.atom.append([ch, coords])

    mol.basis = 'def2-tzvp'
    print(total_elec - num_elec)
    mol.charge = total_elec - num_elec
#     mol.nelectron = num_elec
    mol.verbose = 4
    mol.build()

    method = "HF"
    if method not in ["CCSD", "HF"]:
        raise NotImplementedError("Method %s not supported." % method)

    deltaZ = com.nuclear_charges*(1-lam_val)
    includeonly = np.arange(len(mol.atom_coords()))


    def add_qmmm(calc, mol, deltaZ):
        mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

        def energy_nuc(self):
            q = mol.atom_charges().astype(np.float)
            q[includeonly] += deltaZ
            return mol.energy_nuc(q)

        mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

        return mf

    if method == "HF":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        dm1_ao = calc.make_rdm1()
        total_energy = calc.e_tot
    if method == "CCSD":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        mycc = pyscf.cc.CCSD(calc).run()
        dm1 = mycc.make_rdm1()
        dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
        total_energy = mycc.e_tot

    # GRIDLESS, as things should be ############################
    # Total energy of SCF run

    print("TOTAL_ENERGY", total_energy)

    # Electronic EPN from electron density
    alch_pot_tmp = []
    for site in includeonly:
        mol.set_rinv_orig_(mol.atom_coords()[site])
#         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
        alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
    alchemical_potentials.append(alch_pot_tmp)
    
    # energy of HOMO and LUMO
    nocc = int((total_elec - mol.charge)/2)
    e_homo.append(calc.mo_energy[nocc-1])
    e_lumo.append(calc.mo_energy[nocc])

Calculating 22
30
System: uname_result(system='Linux', node='pc-avl24', release='4.15.0-108-generic', version='#109-Ubuntu SMP Fri Jun 19 11:33:10 UTC 2020', machine='x86_64', processor='x86_64')  Threads 8
Python 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
numpy 1.16.4  scipy 1.2.1
Date: Wed Jul  8 00:11:18 2020
PySCF version 1.7.1
PySCF path  /home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 12
[INPUT] num. electrons = 22
[INPUT] charge = 30
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 C     -0.136883959200   1.448579793000   0.011368940800 AA   -0.258673193734   2.737419078349   0.021484184438 Bohr
[INPUT]  2 C      0.064357729200  -0.043732595200   0.001817971000 AA    0.121618482187  -0.082642627644   0.003435467292 Bohr
[INPUT]  3 O      1.114424575900  -0.629340888600  -0.012303871000 AA    2.105957234936  

/home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf/gto/mole.py:1047: UserWarning: Function mol.dumps drops attribute charge because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= -476.232297958321  delta_E=  279  |g|= 3.42  |ddm|= 26.9
  HOMO = -21.1962908465533  LUMO = -21.1341803782287
cycle= 2 E= -470.273257231361  delta_E= 5.96  |g|=  3.6  |ddm|= 24.8
  HOMO = -21.351476047267  LUMO = -21.0330670059225
cycle= 3 E= -476.76932047857  delta_E= -6.5  |g|= 1.25  |ddm|= 18.9
  HOMO = -21.064181701863  LUMO = -20.8535373650633
cycle= 4 E= -477.50413013686  delta_E= -0.735  |g|= 1.15  |ddm|= 12.6
  HOMO = -21.1420261703447  LUMO = -20.8548316919976
cycle= 5 E= -477.684460264581  delta_E= -0.18  |g|= 0.532  |ddm|= 2.45
  HOMO = -21.2221584528292  LUMO = -20.7886564972623
cycle= 6 E= -477.708576605056  delta_E= -0.0241  |g|= 0.324  |ddm|= 1.29
  HOMO = -21.208344793788  LUMO = -20.8055127903227
cycle= 7 E= -477.721054284092  delta_E= -0.0125  |g|= 0.216  |ddm|= 0.584
  HOMO = -21.2102626698197  LUMO = -20.8227330011623
cycle= 8 E= -477.738618698482  delta_E= -0.0176  |g|= 0.0209  |ddm|= 1.78
  HOMO = -21.2176135064882  LUMO = -20.8082692667534
cycle= 9 E=

number of shells = 97
number of NR pGTOs = 362
number of NR cGTOs = 247
basis = def2-tzvp
ecp = {}
CPU time:      1249.73


******** <class 'pyscf.qmmm.itrf.qmmm_for_scf.<locals>.QMMM'> ********
method = QMMM-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/misa/git_repositories/APDFT/prototyping/atomic_energies/analysis/tmpqihjdv5e
max_memory 4000 MB (current use 252 MB)
** Add background charges for <class 'pyscf.scf.hf.RHF'> **
Set gradient conv threshold to 3.16228e-05
init E= -567.151800541873
  HOMO = -5.42859700297216  LUMO = -5.29041718513667
cycle= 1 E= -511.71696896953  delta_E= 55.4  |g|=  5.1  |ddm|= 15.3
  HOMO = -8.45396196948731  LUMO = -8.39443112848754
cycle= 2 E= -437.676671739582  delta_E=   74  |g|= 10.7  |ddm|= 59.1
  HOMO = -8.179950

  HOMO = -4.45003562537031  LUMO = -4.27656687404613
cycle= 6 E= -468.020547514408  delta_E= -0.224  |g|= 0.543  |ddm|= 0.974
  HOMO = -4.44800112122049  LUMO = -4.24067734702969
cycle= 7 E= -468.078668898194  delta_E= -0.0581  |g|= 0.191  |ddm|= 0.41
  HOMO = -4.48845501603014  LUMO = -4.21484315527763
cycle= 8 E= -468.103254953926  delta_E= -0.0246  |g|= 0.099  |ddm|= 0.334
  HOMO = -4.4932435556934  LUMO = -4.21052985259853
cycle= 9 E= -468.109923320056  delta_E= -0.00667  |g|= 0.121  |ddm|= 0.197
  HOMO = -4.49760413472929  LUMO = -4.20839620990532
cycle= 10 E= -468.114533395654  delta_E= -0.00461  |g|= 0.111  |ddm|= 0.164
  HOMO = -4.49536847258209  LUMO = -4.20738609023438
cycle= 11 E= -468.121212744303  delta_E= -0.00668  |g|=  0.1  |ddm|= 0.223
  HOMO = -4.48126911244142  LUMO = -4.21760272831848
cycle= 12 E= -468.126792016574  delta_E= -0.00558  |g|= 0.0459  |ddm|= 0.212
  HOMO = -4.47847223254293  LUMO = -4.22868896508218
cycle= 13 E= -468.130466651228  delta_E= -0.00367  |g|

cycle= 10 E= -358.701270379051  delta_E= -4.97e-08  |g|= 8.39e-05  |ddm|= 0.00062
  HOMO = -0.470014715741994  LUMO = 0.117422711744371
cycle= 11 E= -358.701270383212  delta_E= -4.16e-09  |g|= 2.76e-05  |ddm|= 0.000189
  HOMO = -0.470018382935327  LUMO = 0.117421290362071
cycle= 12 E= -358.701270383614  delta_E= -4.03e-10  |g|= 1.04e-05  |ddm|= 4.6e-05
  HOMO = -0.470018013000769  LUMO = 0.117421878411601
Extra cycle  E= -358.701270383675  delta_E= -6.12e-11  |g|= 7.28e-06  |ddm|= 1.18e-05
converged SCF energy = -358.701270383675
TOTAL_ENERGY -358.7012703836754


In [82]:
calc.get_occ()

NPArrayWithTag([2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
alchemical_potentials

In [ ]:
e_homo

In [ ]:
e_lumo

In [ ]:
base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
save_obj(alchemical_potentials, base+'alch_pots')
save_obj(e_homo, base+'e_homo')
save_obj(e_lumo, base+'e_lumo')

### Plot results

In [67]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import qml
from ase.units import Bohr

import sys
sys.path.insert(0,'/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import alchemy_tools2 as at
import utils_qm as ut

In [69]:
# load data
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')

base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
alchs = ut.load_obj(base+'alch_pots')
alch_pots = np.array(alchs)
alch_pots = np.concatenate((np.array([np.zeros(12)]), alch_pots))
homos = ut.load_obj(base+'e_homo')
homos.insert(0,0.0)
homos = np.array(homos)

lumos = ut.load_obj(base+'e_lumo')
lumos.insert(0,0.0)
lumos = np.array(lumos)

lam_vals = np.array([0, 22, 30, 38, 44, 52])/52

In [ ]:
# # for guido
# np.savetxt('/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/alch_pots.txt', alch_pots)
# np.savetxt('/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/e_homos.txt', homos)
# np.savetxt('/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/lam_vals.txt', lam_vals)

### Calculation of atomic energies

integrate $\mu_I$ over lambda

In [70]:
alch_pots_integrated = []
for i in range(alch_pots.shape[1]):
    alch_pots_integrated.append(np.trapz(alch_pots[:,i], x=lam_vals, axis=0))

In [71]:
alch_pots_integrated

[11.168975067718874,
 12.778729741277402,
 14.208595094759858,
 15.110048524424798,
 12.262530086247827,
 11.622888392967564,
 11.74960303099666,
 5.387493461248392,
 5.54044002867333,
 5.540518135874408,
 6.366716972681599,
 6.366539524560403]

multiply by nuclear charge

In [72]:
atomic_energies_without_rep = -(alch_pots_integrated*com.nuclear_charges)

In [73]:
atomic_energies_without_rep.sum()

-632.996818022762

subtract nuclear repulsion

In [74]:
nuc_rep_atomic = []
for i in lam_vals:
    nucs = i*com.nuclear_charges
    nuc_rep_atomic.append(at.nuclear_repulsion(nucs, com.coordinates/Bohr))

nuc_rep_atomic[-1]

array([28.69240698, 38.75272775, 37.98407444, 45.24721636, 35.22425798,
       31.84765518, 27.58435185,  4.99136815,  5.15367162,  5.15365186,
        5.9496209 ,  5.94944644])

In [76]:
nuc_rep_atomic[-1].sum() + atomic_energies_without_rep.sum()

-360.46636850108456

In [78]:
homo = 52*np.trapz(homos, x=lam_vals, axis=0)
lumo = 52*np.trapz(lumos, x=lam_vals, axis=0)

In [79]:
nuc_rep_atomic[-1].sum() + atomic_energies_without_rep.sum() + (homo + lumo)/2

-367.21487070143155

In [ ]:
for alch, nuc_charge in alch_pots:

calculation of $\sum_I \mu_I(\lambda)$, $\sum_I E_I(\lambda)$ and $\lambda$

In [ ]:
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
total_elec = com.nuclear_charges.sum()

alch_pots_sum = []
atomic_energies_sum = []
for i in alch_pots:
    atomic_energies_sum.append((i*com.nuclear_charges).sum())
    print((i*com.nuclear_charges).sum())
    alch_pots_sum.append(i.sum())

alch_pots_sum.insert(0,0.0)  
atomic_energies_sum.insert(0,0.0) 


calculation of atomic nuclear repulsion at every lambda $E_{I, \rm{NN}}(\lambda)$ and atomic energy with repulsion

In [ ]:
nuc_rep_atomic = []
for i in lam_vals:
    nucs = i*com.nuclear_charges
    nuc_rep_atomic.append(at.nuclear_repulsion(nucs, com.coordinates/Bohr))

In [ ]:
atomic_en_with_nuc_rep = []
for alch, nuc_rep, nuc_charge in zip(alch_pots, nuc_rep_atomic):
    atomic_minus_nu.append(alch-nuc)


In [ ]:
sum_at = []
for i in atomic_minus_nu:
    sum_at.append(i.sum())

In [ ]:
np.array(sum_at).sum()

In [ ]:
np.trapz(sum_at, x=lam_vals, axis=0)

In [ ]:
fig, ax = plt.subplots(1,1)
plt.rcParams['font.size'] = 18
ax.plot(lam_vals, alch_pots_sum, '-o', label='$\sum_I \mu_I$')
ax.plot(lam_vals, atomic_energies_sum, '-o', label='$\sum_I Z_I \mu_I$')

ax.legend()
ax.set_ylabel('$\sum_I \mathcal{P}_I$')
ax.set_xlabel('$\lambda$')
# plt.savefig('/home/misa/projects/Atomic-Energies/figures/different_resevoir/com_001212_alch_pot_atomic_en.png', bbox_inches='tight')

- Why is integral over not equal to total energy at lambda=1? Is the nuc-nuc repulsion included?
What is the expression for $E(\lambda)$, what is the integral over the whole path?
- When/How to add e homo? For every lambda or only lambda = 1?
- Are there still some bugs in the code?
- Are the EPN's equal to the alchemical potential or the atomic energy?
- Calculate some lower values for lambda?

Plots vs $\lambda$:
- $E(\lambda)$ 
- $\sum_I \mu_I(\lambda)$
- $\sum_I E_I(\lambda)$
- $\int_0^\lambda E_I(\lambda') d\lambda'$

In [ ]:
# com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
# total_elec = com.nuclear_charges.sum()
# total_num_ve = 38
# num_ves = [8, 16, 24, 30, 38]
# alchemical_potentials = []
# e_homo = []
# e_lumo = []

# for num_ve in num_ves:
#     print(f'Calculating {num_ve}')
#     lam_val = num_ve/total_num_ve
#     # core_elec = com.nuclear_charges.sum() - total_num_ve

#     mol = pyscf.gto.Mole()
#     for ch, coords in zip(com.nuclear_charges, com.coordinates):
#         mol.atom.append([ch, coords])

#     mol.basis = 'def2-tzvp'
#     mol.charge = total_num_ve - num_ve
#     mol.verbose = 0
#     mol.build()

#     method = "HF"
#     if method not in ["CCSD", "HF"]:
#         raise NotImplementedError("Method %s not supported." % method)

#     deltaZ = -com.nuclear_charges*(1-lam_val)
#     includeonly = np.arange(len(mol.atom_coords()))


#     def add_qmmm(calc, mol, deltaZ):
#         mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

#         def energy_nuc(self):
#             q = mol.atom_charges().astype(np.float)
#             q[includeonly] += deltaZ
#             return mol.energy_nuc(q)

#         mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

#         return mf

#     if method == "HF":
#         calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
#         hfe = calc.kernel(verbose=0)
#         dm1_ao = calc.make_rdm1()
#         total_energy = calc.e_tot
#     if method == "CCSD":
#         calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
#         hfe = calc.kernel(verbose=0)
#         mycc = pyscf.cc.CCSD(calc).run()
#         dm1 = mycc.make_rdm1()
#         dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
#         total_energy = mycc.e_tot

#     # GRIDLESS, as things should be ############################
#     # Total energy of SCF run

#     print("TOTAL_ENERGY", total_energy)

#     # Electronic EPN from electron density
#     alch_pot_tmp = []
#     for site in includeonly:
#         mol.set_rinv_orig_(mol.atom_coords()[site])
# #         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
#         alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
#     alchemical_potentials.append(alch_pot_tmp)
    
#     # energy of HOMO and LUMO
#     nocc = int((total_elec - mol.charge)/2)
#     e_homo.append(calc.mo_energy[nocc-1])
#     e_lumo.append(calc.mo_energy[nocc])

### Test N2 from alchemy

- why is mean of lumo, homo better?
- how good are atomisation energies? (error canceling)
- how to calcs for unpaired electrons?

In [1]:
import numpy as np
import scipy as sc
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.data import nist
import qml

angstrom = 1 / 0.52917721067

In [54]:
def N2_diatomics():
    el = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N']
    energies = []
    alchemical_potentials = []
    e_homos = []
    e_lumos = []
    for e in el:
        mol = pyscf.gto.Mole()
        mol.atom = f'{e} 0 0 0; {e} 0 0 1.5'
        mol.basis = 'def2-tzvp'
        mol.verbose = 0
        mol.build()
        calc = pyscf.scf.RHF(mol)
        calc.kernel()
        energies.append(calc.e_tot)
        dm1_ao = calc.make_rdm1()
        # get alchemical potential
        # Electronic EPN from electron density
        alch_pot_tmp = []
        for site in [0,1]:
            mol.set_rinv_orig_(mol.atom_coords()[site])
    #         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
            alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
        alchemical_potentials.append(alch_pot_tmp)

        # get ehomo
        idx_homo = np.where(calc.get_occ()>0)[0][-1]
        idx_lumo = idx_homo + 1
        calc.mo_energy.sort()
        e_homos.append(calc.mo_energy[idx_homo])
        e_lumos.append(calc.mo_energy[idx_lumo])
        
    alchemical_potentials.insert(0, [0.0, 0.0])
    alchemical_potentials = np.array(alchemical_potentials)*(-1)
    e_homos.insert(0, 0)
    e_homos = np.array(e_homos)
    e_lumos.insert(0, 0)
    e_lumos = np.array(e_lumos)
    
    return(alchemical_potentials, e_homos, e_lumos, calc.energy_nuc())

def N_atoms():
    el = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N']
    energies = []
    alchemical_potentials = []
    e_homos = []
    e_lumos = []
    for e, Z in zip(el, np.arange(1,8)):
        mol = pyscf.gto.Mole()
        mol.atom = f'{e} 0 0 0'
        mol.basis = 'def2-tzvp'
        mol.verbose = 0
        mol.spin = Z%2
        mol.build()
        calc = pyscf.scf.RHF(mol)
        calc.kernel()
        energies.append(calc.e_tot)
        dm1_ao = calc.make_rdm1()
        # get alchemical potential
        # Electronic EPN from electron density
        alch_pot_tmp = []
        for site in [0]:
            mol.set_rinv_orig_(mol.atom_coords()[site])
    #         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
            alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
        alchemical_potentials.append(alch_pot_tmp)

        # get ehomo
        idx_homo = np.where(calc.get_occ()>0)[0][-1]
        idx_lumo = idx_homo + 1
        calc.mo_energy.sort()
        e_homos.append(calc.mo_energy[idx_homo])
        e_lumos.append(calc.mo_energy[idx_lumo])
        
    alchemical_potentials.insert(0, [0.0, 0.0])
    alchemical_potentials = np.array(alchemical_potentials)*(-1)
    e_homos.insert(0, 0)
    e_homos = np.array(e_homos)
    e_lumos.insert(0, 0)
    e_lumos = np.array(e_lumos)
    
    return(alchemical_potentials, e_homos, e_lumos, calc.energy_nuc())

def N2_APDFT():
    total_elec = 14
    # num_elecs = np.arange(15)
    num_elecs = [2, 4, 6, 8, 10, 12, 14]

    alchemical_potentials = []
    e_homos = []
    e_lumos = []

    for num_elec in num_elecs:
        lam_val = num_elec/total_elec

        mol = pyscf.gto.Mole()
        mol.atom = 'N 0 0 0; N 0 0 1.5'
        mol.basis = 'def2-tzvp'
        print(total_elec - num_elec)
        mol.charge = total_elec - num_elec
#         mol.spin = num_elec%2
        mol.verbose = 0
        mol.build()

        method = "HF"
        if method not in ["CCSD", "HF"]:
            raise NotImplementedError("Method %s not supported." % method)

        deltaZ = [-mol.charge/2, -mol.charge/2]
        includeonly = np.arange(len(mol.atom_coords()))


        def add_qmmm(calc, mol, deltaZ):
            mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

            def energy_nuc(self):
                q = mol.atom_charges().astype(np.float)
                q[includeonly] += deltaZ
                return mol.energy_nuc(q)

            mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

            return mf

        if method == "HF":
            calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
            hfe = calc.kernel(verbose=0)
            dm1_ao = calc.make_rdm1()
            total_energy = calc.e_tot
        if method == "CCSD":
            calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
            hfe = calc.kernel(verbose=0)
            mycc = pyscf.cc.CCSD(calc).run()
            dm1 = mycc.make_rdm1()
            dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
            total_energy = mycc.e_tot

        # GRIDLESS, as things should be ############################
        # Total energy of SCF run

        print("TOTAL_ENERGY", total_energy)

        # Electronic EPN from electron density
        alch_pot_tmp = []
        for site in includeonly:
            mol.set_rinv_orig_(mol.atom_coords()[site])
    #         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
            alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
        alchemical_potentials.append(alch_pot_tmp)

        # energy of HOMO and LUMO
        nocc = int((total_elec - mol.charge)/2)
        e_homos.append(calc.mo_energy[nocc-1])
        e_lumos.append(calc.mo_energy[nocc])
        
    alchemical_potentials.insert(0, [0.0, 0.0])
    alchemical_potentials = np.array(alchemical_potentials)*(-1)
    e_homos.insert(0, 0)
    e_homos = np.array(e_homos)
    e_lumos.insert(0, 0)
    e_lumos = np.array(e_lumos)
        
    return(alchemical_potentials, e_homos, e_lumos, calc.energy_nuc())
    
def get_atomic_energy(lam_vals, alchemical_potentials, nuc_rep, e_homos, dZ, dN):
    mu_I = np.trapz(alchemical_potentials, lam_vals)
    e_homo = np.trapz(e_homos, lam_vals)
    
    return(mu_I * dZ + e_homo*dN + nuc_rep)

### Referenz calculation

In [44]:
mol = pyscf.gto.Mole()
mol.atom = 'N 0 0 0; N 0 0 1.5'
mol.basis = 'def2-tzvp'
mol.build()
m = pyscf.scf.RHF(mol)
m.kernel()
e_ref = m.e_tot

converged SCF energy = -108.706811596033


### N$_2$ from diatomics

In [24]:
alchemical_pots, ehomos, elumos, e_nuc = N2_diatomics()

In [46]:
lam_vals = np.arange(8)*2/14

e_from_homo = get_atomic_energy(lam_vals, alchemical_pots[:,0], e_nuc/2, ehomos, 7, 7)*2
e_from_lumo = get_atomic_energy(lam_vals, alchemical_pots[:,0], e_nuc/2, elumos, 7, 7)*2

mean = (e_from_homo + e_from_lumo)/2

print(f'energy using homo = {e_from_homo}, delta E_ref = {e_from_homo - e_ref}')
print(f'energy using lumo = {e_from_lumo}, delta E_ref = {e_from_lumo - e_ref}')
print(f'energy using mean of homo and lumo = {mean}, delta E_ref = {mean - e_ref}')

energy using homo = -112.26711496349947, delta E_ref = -3.5603033674666307
energy using lumo = -105.41372728907514, delta E_ref = 3.293084306957695
energy using mean of homo and lumo = -108.84042112628731, delta E_ref = -0.13360953025447486


### N$_2$ from diatomics with APDFT implementation

In [37]:
alchemical_pots2, ehomos2, elumos2, e_nuc2 = N2_APDFT()

12
TOTAL_ENERGY -1.0011014198648525
10
TOTAL_ENERGY -5.6285244596173225
8
TOTAL_ENERGY -14.393211474040479
6
TOTAL_ENERGY -28.563023109075985
4
TOTAL_ENERGY -48.756212814630295
2
TOTAL_ENERGY -75.26768502575823
0
TOTAL_ENERGY -108.70681159603274


In [47]:
lam_vals = np.arange(8)*2/14
e_from_homo2 = get_atomic_energy(lam_vals, alchemical_pots2[:,0], e_nuc2/2, ehomos2, 7, 7)*2
e_from_lumo2 = get_atomic_energy(lam_vals, alchemical_pots2[:,0], e_nuc2/2, elumos2, 7, 7)*2
mean2 = (e_from_homo2 + e_from_lumo2)/2

print(f'energy using homo = {e_from_homo2}, delta E_ref = {e_from_homo2 - e_ref}')
print(f'energy using lumo = {e_from_lumo2}, delta E_ref = {e_from_lumo2 - e_ref}')
print(f'energy using mean of homo and lumo = {mean2}, delta E_ref = {mean2 - e_ref}')

energy using homo = -112.49942186758364, delta E_ref = -3.7926102715507994
energy using lumo = -106.63758746154318, delta E_ref = 2.0692241344896587
energy using mean of homo and lumo = -109.5685046645634, delta E_ref = -0.8616930685305704


### Atomisation energy

/home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf/gto/mole.py:1047: UserWarning: Function mol.dumps drops attribute spin because it is not JSON-serializable
  warnings.warn(msg)


(array([list([]), list([]), list([]), list([]), list([]), list([]),
        list([]), list([])], dtype=object),
 array([ 0.        , -0.49980983, -0.91687015, -0.07925089, -0.30920635,
        -0.08587755, -0.35719238, -0.2026544 ]),
 array([0.        , 0.02708109, 0.82957977, 0.07829559, 0.05813838,
        0.04628395, 0.03382565, 0.02177696]),
 0)

In [65]:
el = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N']
energies = []
alchemical_potentials = []
e_homos = []
e_lumos = []

mol = pyscf.gto.Mole()
mol.atom = f'li 0 0 0'
mol.basis = 'def2-tzvp'
mol.verbose = 0
mol.spin = 1
mol.build()
calc = pyscf.scf.RHF(mol)
calc.kernel()
energies.append(calc.e_tot)
dm1_ao = calc.make_rdm1()
# get alchemical potential
# Electronic EPN from electron density
alch_pot_tmp = []
mol.set_rinv_orig_(mol.atom_coords()[0])
#         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())

# get ehomo
idx_homo = np.where(calc.get_occ()>0)[0][-1]
idx_lumo = idx_homo + 1
calc.mo_energy.sort()
e_homos.append(calc.mo_energy[idx_homo])
e_lumos.append(calc.mo_energy[idx_lumo])

In [66]:
alch_pot_tmp

[array([3.04307909, 2.15391106, 1.24098971, 0.3789949 , 0.18517513,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ])]